In [ ]:
# raw_feriados
# Este notebook cria a base de feriados considerando os nacionais, facultativos, estaduais e municipais e também os associa a suas localizações

In [1]:
%run ../spark-default.py

In [2]:
from datetime import date, timedelta
from unidecode import unidecode
import builtins as b

In [3]:
dates = []

weekdays = {0: "SEG", 1: "TER", 2: "QUA", 3: "QUI", 4: "SEX", 5: "SAB", 6: "DOM"}

d = date(2010, 1, 1)

while True:
    o = {
        "date": d.strftime("%Y-%m-%d"),
        "year": d.year,
        "month": d.month,
        "day": d.day,
        "dow": weekdays[d.weekday()]
    }
    dates.append(o)

    if (d.year >= 2025):
        break
    else:
        d += timedelta(days=1)

dates = spark.createDataFrame(dates)
dates.show()

+----------+---+---+-----+----+
|      date|day|dow|month|year|
+----------+---+---+-----+----+
|2010-01-01|  1|SEX|    1|2010|
|2010-01-02|  2|SAB|    1|2010|
|2010-01-03|  3|DOM|    1|2010|
|2010-01-04|  4|SEG|    1|2010|
|2010-01-05|  5|TER|    1|2010|
|2010-01-06|  6|QUA|    1|2010|
|2010-01-07|  7|QUI|    1|2010|
|2010-01-08|  8|SEX|    1|2010|
|2010-01-09|  9|SAB|    1|2010|
|2010-01-10| 10|DOM|    1|2010|
|2010-01-11| 11|SEG|    1|2010|
|2010-01-12| 12|TER|    1|2010|
|2010-01-13| 13|QUA|    1|2010|
|2010-01-14| 14|QUI|    1|2010|
|2010-01-15| 15|SEX|    1|2010|
|2010-01-16| 16|SAB|    1|2010|
|2010-01-17| 17|DOM|    1|2010|
|2010-01-18| 18|SEG|    1|2010|
|2010-01-19| 19|TER|    1|2010|
|2010-01-20| 20|QUA|    1|2010|
+----------+---+---+-----+----+
only showing top 20 rows



In [4]:
schema = StructType([
    StructField("data", StringType(), True),
    StructField("nome", StringType(), True),
    StructField("tipo", StringType(), True),
    StructField("descricao", StringType(), True),
    StructField("uf", StringType(), True),
    StructField("municipio", StringType(), True)
])

def fix_data(obj):
    obj = obj.asDict()
    data = obj["data"].split("/")
    municipio = None if obj["municipio"] == " " else obj["municipio"]
    return {
        **obj,
        "data": data[2] + "-" + data[1] + "-" + data[0],
        "municipio": municipio
    }

In [5]:
feriados_nacionais = (spark.read
                      .schema(schema)
                      .csv("s3a://datalake/source/feriados/nacional/", sep=",", escape='"')
                      .rdd
                      .map(fix_data)
                      .toDF(schema=schema)
                     )

feriados_nacionais.show()

+----------+--------------------+--------+--------------------+----+---------+
|      data|                nome|    tipo|           descricao|  uf|municipio|
+----------+--------------------+--------+--------------------+----+---------+
|2010-01-01|            Ano Novo|NACIONAL|O Ano-Novo ou Rév...|null|     null|
|2010-04-02|   Sexta-Feira Santa|NACIONAL|Também chamada de...|null|     null|
|2010-04-21|   Dia de Tiradentes|NACIONAL|Joaquim José da S...|null|     null|
|2010-05-01|     Dia do Trabalho|NACIONAL|O Dia do Trabalha...|null|     null|
|2010-09-07|Independência do ...|NACIONAL|O Dia da Indepênc...|null|     null|
|2010-10-12|Nossa Senhora Apa...|NACIONAL|Nossa Senhora da ...|null|     null|
|2010-11-02|      Dia de Finados|NACIONAL|O Dia de Finados ...|null|     null|
|2010-11-15|Proclamação da Re...|NACIONAL|A Proclamação da ...|null|     null|
|2010-12-25|               Natal|NACIONAL|O Natal é comemor...|null|     null|
|2011-01-01|            Ano Novo|NACIONAL|O Ano-Novo

In [6]:
feriados_facultativos = (spark.read
                      .schema(schema)
                      .csv("s3a://datalake/source/feriados/facultativo/", sep=",", escape='"')
                      .rdd
                      .map(fix_data)
                      .toDF(schema=schema)
                     )

feriados_facultativos.show()

+----------+--------------------+-----------+--------------------+----+---------+
|      data|                nome|       tipo|           descricao|  uf|municipio|
+----------+--------------------+-----------+--------------------+----+---------+
|2019-03-04|            Carnaval|FACULTATIVO|Ponto Facultativo...|null|     null|
|2019-03-05|            Carnaval|FACULTATIVO|Carnaval NÃO é um...|null|     null|
|2019-03-06|            Carnaval|FACULTATIVO|Ponto Facultativo...|null|     null|
|2019-06-20|      Corpus Christi|FACULTATIVO|Ponto Facultativo...|null|     null|
|2019-10-15|    Dia do Professor|FACULTATIVO|FERIADO ESCOLAR -...|null|     null|
|2019-10-28|Dia do Servidor P...|FACULTATIVO|O dia do servidor...|null|     null|
|2019-10-28|Ponto Facultativo...|FACULTATIVO|Dia do Servidor P...|null|     null|
|2019-03-29|Aniversário da Ci...|FACULTATIVO|Fundação de Curit...|null|     null|
|2019-12-24|         Facultativo|FACULTATIVO|    Véspera do Natal|null|     null|
|2019-11-20|Dia 

In [7]:
feriados_estaduais = (spark.read
                      .schema(schema)
                      .csv("s3a://datalake/source/feriados/estadual/", sep=",", escape='"')
                      .rdd
                      .map(fix_data)
                      .toDF(schema=schema)
                     )

feriados_estaduais.show()

+----------+--------------------+--------+--------------------+---+---------+
|      data|                nome|    tipo|           descricao| uf|municipio|
+----------+--------------------+--------+--------------------+---+---------+
|2015-01-20|     Dia do Católico|ESTADUAL|Dia 20 de janeiro...| AC|     null|
|2015-01-23|   Dia do Evangélico|ESTADUAL|Dia 23 de janeiro...| AC|     null|
|2015-03-08|       Dia da Mulher|ESTADUAL|Feriado conforme ...| AC|     null|
|2015-06-15| Aniversário do Acre|ESTADUAL|De acordo com a L...| AC|     null|
|2015-08-06|   Revolução Acreana|ESTADUAL|                null| AC|     null|
|2015-09-05|     Dia da Amazônia|ESTADUAL|Conforme a lei Nº...| AC|     null|
|2015-06-24|     Dia de São João|ESTADUAL|O mastro de São J...| AL|     null|
|2015-06-29|    Dia de São Pedro|ESTADUAL|São Pedro foi um ...| AL|     null|
|2015-09-16|Emancipação de Al...|ESTADUAL|Comemoração da em...| AL|     null|
|2015-11-20|Dia da Consciênci...|ESTADUAL|O Dia Nacional da...| 

In [8]:
feriados_municipais = (spark.read
                      .schema(schema)
                      .option("multiline", "true")
                      .csv("s3a://datalake/source/feriados/municipal/", sep=",", escape='"')
                      .rdd
                      .map(fix_data)
                      .toDF(schema=schema)
                     )

feriados_municipais.show()

+----------+--------------------+---------+--------------------+---+----------------+
|      data|                nome|     tipo|           descricao| uf|       municipio|
+----------+--------------------+---------+--------------------+---+----------------+
|2014-04-28|   Feriado Municipal|MUNICIPAL|                null| AC|     Acrelândia |
|2014-09-06|   Feriado Municipal|MUNICIPAL|                null| AC|     Acrelândia |
|2014-05-14|   Feriado Municipal|MUNICIPAL|                null| AC|   Assis Brasil |
|2014-05-31|   Feriado Municipal|MUNICIPAL|                null| AC|   Assis Brasil |
|2014-08-15|   Feriado Municipal|MUNICIPAL|                null| AC|   Assis Brasil |
|2014-07-03|   Feriado Municipal|MUNICIPAL|                null| AC|      Brasiléia |
|2014-09-06|   Feriado Municipal|MUNICIPAL|                null| AC|      Brasiléia |
|2014-09-06|   Feriado Municipal|MUNICIPAL|                null| AC|         Bujari |
|2014-09-06|   Feriado Municipal|MUNICIPAL|           

In [9]:
feriados = (feriados_estaduais
            .union(feriados_facultativos)
            .union(feriados_municipais)
            .union(feriados_nacionais)
            .withColumn("municipio", when(col("municipio") == lit(" "), lit(None)).when(col("municipio") == lit(""), lit(None)).otherwise(col("municipio")))
           )

print(feriados.count())
feriados.show()

138214
+----------+--------------------+--------+--------------------+---+---------+
|      data|                nome|    tipo|           descricao| uf|municipio|
+----------+--------------------+--------+--------------------+---+---------+
|2015-01-20|     Dia do Católico|ESTADUAL|Dia 20 de janeiro...| AC|     null|
|2015-01-23|   Dia do Evangélico|ESTADUAL|Dia 23 de janeiro...| AC|     null|
|2015-03-08|       Dia da Mulher|ESTADUAL|Feriado conforme ...| AC|     null|
|2015-06-15| Aniversário do Acre|ESTADUAL|De acordo com a L...| AC|     null|
|2015-08-06|   Revolução Acreana|ESTADUAL|                null| AC|     null|
|2015-09-05|     Dia da Amazônia|ESTADUAL|Conforme a lei Nº...| AC|     null|
|2015-06-24|     Dia de São João|ESTADUAL|O mastro de São J...| AL|     null|
|2015-06-29|    Dia de São Pedro|ESTADUAL|São Pedro foi um ...| AL|     null|
|2015-09-16|Emancipação de Al...|ESTADUAL|Comemoração da em...| AL|     null|
|2015-11-20|Dia da Consciênci...|ESTADUAL|O Dia Nacional 

In [10]:
municipios = spark.read.option("header", True).option("sep", ";").option("encoding", "ISO-8859-1").csv("s3a://datalake/source/municipios_csv/")

municipios.printSchema()

municipios = municipios.select(
    col("UF").alias("uf"),
    col("Nome_UF").alias("nome_uf"),
    col("Município").alias("municipio"),
    col("Código Município Completo").alias("cod_municipio_completo"),
    col("Nome_Município").alias("nome_municipio"),
    substring('Código Município Completo', 1,6).alias("cod_municipio")
)

print(municipios.count())

municipios.show(truncate=False)

root
 |-- UF: string (nullable = true)
 |-- Nome_UF: string (nullable = true)
 |-- Região Geográfica Intermediária: string (nullable = true)
 |-- Nome Região Geográfica Intermediária: string (nullable = true)
 |-- Região Geográfica Imediata: string (nullable = true)
 |-- Nome Região Geográfica Imediata: string (nullable = true)
 |-- Mesorregião Geográfica: string (nullable = true)
 |-- Nome_Mesorregião: string (nullable = true)
 |-- Microrregião Geográfica: string (nullable = true)
 |-- Nome_Microrregião: string (nullable = true)
 |-- Município: string (nullable = true)
 |-- Código Município Completo: string (nullable = true)
 |-- Nome_Município: string (nullable = true)

5570
+---+--------+---------+----------------------+-------------------------+-------------+
|uf |nome_uf |municipio|cod_municipio_completo|nome_municipio           |cod_municipio|
+---+--------+---------+----------------------+-------------------------+-------------+
|11 |Rondônia|00015    |1100015               |Alt

In [11]:
def remover_acentos(texto):
    return unidecode(texto).lower() if texto else None

remover_acentos_udf = udf(remover_acentos, StringType())

In [16]:
final = (
    feriados
        .withColumn("municipio_formatado", trim(remover_acentos_udf(col("municipio"))))
        .alias("f")
        .join(
            municipios
                .withColumn("nome_municipio_formatado", trim(remover_acentos_udf(col("nome_municipio"))))
                .alias("c"), 
            col("f.municipio_formatado") == col("c.nome_municipio_formatado"), "left")
        .select(
            "f.data",
            "f.nome",
            "f.tipo",
            "f.descricao",
            "f.uf",
            "f.municipio",
            "c.cod_municipio"
        )
)

final.filter("f.municipio is not null").show()

+----------+--------------------+---------+--------------------+---+----------------+-------------+
|      data|                nome|     tipo|           descricao| uf|       municipio|cod_municipio|
+----------+--------------------+---------+--------------------+---+----------------+-------------+
|2014-04-28|   Feriado Municipal|MUNICIPAL|                null| AC|     Acrelândia |       120001|
|2014-09-06|   Feriado Municipal|MUNICIPAL|                null| AC|     Acrelândia |       120001|
|2014-05-14|   Feriado Municipal|MUNICIPAL|                null| AC|   Assis Brasil |       120005|
|2014-05-31|   Feriado Municipal|MUNICIPAL|                null| AC|   Assis Brasil |       120005|
|2014-08-15|   Feriado Municipal|MUNICIPAL|                null| AC|   Assis Brasil |       120005|
|2014-07-03|   Feriado Municipal|MUNICIPAL|                null| AC|      Brasiléia |       120010|
|2014-09-06|   Feriado Municipal|MUNICIPAL|                null| AC|      Brasiléia |       120010|


In [18]:
# Check for erros...

print(
    final.filter("f.municipio is not null and cod_municipio is null").count()
)

final.filter("f.municipio is not null and cod_municipio is null").show()

543
+----------+--------------------+---------+---------------+---+--------------------+-------------+
|      data|                nome|     tipo|      descricao| uf|           municipio|cod_municipio|
+----------+--------------------+---------+---------------+---+--------------------+-------------+
|2014-07-20|   Feriado Municipal|MUNICIPAL|           null| CE|            Itapagé |         null|
|2014-08-07|   Feriado Municipal|MUNICIPAL|           null| MG|         Brasópolis |         null|
|2014-09-16|   Feriado Municipal|MUNICIPAL|           null| MG|         Brasópolis |         null|
|2014-12-08|   Feriado Municipal|MUNICIPAL|           null| MG|         Brasópolis |         null|
|2014-06-24|   Feriado Municipal|MUNICIPAL|           null| MT|            Poxoréo |         null|
|2014-05-05|   Feriado Municipal|MUNICIPAL|           null| MT|Santo Antônio do ...|         null|
|2014-06-13|Dia de Santo Antônio|MUNICIPAL|           null| MT|Santo Antônio do ...|         null|
|2014-

In [19]:
final.write.mode("overwrite").parquet("s3a://datalake/raw/feriados/")